In [112]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from scipy import signal
from pmdarima.arima import auto_arima

In [113]:
df = pd.read_csv("../Data/Final/STI0220_cov20_pop0220.csv", index_col = 0)
df.head()

,year,fips,cases_chlamydia,cases_early_non-primary_non-secondary_syphilis,cases_gonorrhea,cases_primary_and_secondary_syphilis,rate_per_100000_chlamydia,rate_per_100000_early_non-primary_non-secondary_syphilis,rate_per_100000_gonorrhea,rate_per_100000_primary_and_secondary_syphilis,...,pct_ind_transportation_and_warehousing_and_utilities,pct_ind_information,pct_ind_finance_and_insurance_and_real_estate_and_rental_and_leasing,pct_ind_professional_scientific_and_management_and_administrative_and_waste_management_services,pct_ind_educational_services_and_health_care_and_social_assistance,pct_ind_arts_entertainment_and_recreation_and_accommodation_and_food_services,pct_ind_other_services_except_public_administration,pct_ind_public_administration,health_insurance_rate,poverty_rate
0,2000,45001,61.0,1.0,57.0,2.0,232.6,3.8,217.4,7.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000,22001,0.0,8.0,0.0,11.0,0.0,13.6,0.0,18.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000,51001,153.0,2.0,88.0,0.0,398.7,5.2,229.3,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000,16001,644.0,0.0,33.0,1.0,212.3,0.0,10.9,0.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000,19001,8.0,0.0,0.0,0.0,97.6,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [115]:
counts = df['fips'].value_counts()
#ys = df[df['fips'].isin(counts[counts == 19].index)]
#ys
counts
#df[df['fips']==30037]

30037    46
48117    46
51133    21
2180     21
51710    21
         ..
72047    11
72045    11
72043    11
2158      6
46102     6
Name: fips, Length: 3214, dtype: int64

In [104]:
cols = ['cases_gonorrhea','cases_chlamydia','cases_early_non-primary_non-secondary_syphilis',
        'cases_primary_and_secondary_syphilis', 'year']

df = df[df['year'] >= 2002]
fips = df['fips'].unique()
print(len(fips))
print(df.dtypes)

3214
year                                                                               int64
fips                                                                               int64
cases_chlamydia                                                                  float64
cases_early_non-primary_non-secondary_syphilis                                   float64
cases_gonorrhea                                                                  float64
                                                                                  ...   
pct_ind_arts_entertainment_and_recreation_and_accommodation_and_food_services    float64
pct_ind_other_services_except_public_administration                              float64
pct_ind_public_administration                                                    float64
health_insurance_rate                                                            float64
poverty_rate                                                                     float64
Length: 180, dty

In [105]:
act_values = []
pred_values = []
errors = []

#fips = fips[:500]

for index, i in enumerate(fips):
    df1 = df[(df['fips']==i) & (df['year']>= 2002)].loc[:,cols]
    df_sum = df1.groupby('year').sum().reset_index()
    df_model = df_sum[["year", "cases_chlamydia"]]
    #print(df_model.shape)
    #df_model['cases_chlamydia'] = signal.detrend(df_model.cases_chlamydia)
    actual_value = df_model.iloc[18, df_model.columns.get_loc('cases_chlamydia')]
    #print(df_model.head(20))
    #print(actual_value)
    df_model = df_model[(df_model["year"] <= 2019)]
    df_plot = df_model
    #print(actual_value)
    
    model = auto_arima(df_model["cases_chlamydia"], trace=False, error_action='ignore', suppress_warnings=True)
    model.fit(df_model["cases_chlamydia"])
    
    pred = model.predict(n_periods = 1)
    fitted_value = pred[0]
    #print(df['year'].nunique())
    
    act_values.append(actual_value)
    pred_values.append(fitted_value)
    errors.append(actual_value - fitted_value)
    
    #print(model_fit.summary())
    if index <= 10:
        print(i , ':    ', actual_value, ' & ', fitted_value)

45001 :     99.0  &  118.07601614330063
22001 :     369.0  &  356.69003051931895
51001 :     210.0  &  252.88083214202737
16001 :     2163.0  &  2288.470588235294
19001 :     24.0  &  21.108468724488453
21001 :     36.0  &  51.4860266956982
29001 :     79.0  &  89.0
40001 :     84.0  &  109.9609579917383
8001 :     2627.0  &  2903.0
19003 :     7.0  &  8.516267642798466
16003 :     4.0  &  4.3888888888888875


IndexError: index 18 is out of bounds for axis 0 with size 18

In [ ]:
dff = pd.DataFrame()
dff['fips'] = fips
dff['pred_values_chlamydia'] = pred_values
dff['errors_chlamydia'] = errors

In [ ]:
act_values = []
pred_values = []
errors = []

#fips = fips[:500]

for index, i in enumerate(fips):
    df1 = df[(df['fips']==i) & (df['year']>= 2002)].loc[:,cols]
    df_sum = df1.groupby('year').sum().reset_index()
    df_model = df_sum[["year", "cases_gonorrhea"]]
    #print(df_model.shape)
    #df_model['cases_gonorrhea'] = signal.detrend(df_model.cases_gonorrhea)
    actual_value = df_model.iloc[, df_model.columns.get_loc('cases_gonorrhea')] 
    df_model = df_model[(df_model["year"] <= 2019)]
    df_plot = df_model
    #print(df_model.head(20))
    #print(actual_value)
    
    model = auto_arima(df_model["cases_gonorrhea"], trace=False, error_action='ignore', suppress_warnings=True)
    model.fit(df_model["cases_gonorrhea"])
    
    pred = model.predict(n_periods = 1)
    fitted_value = pred[0]
    #print(df['year'].nunique())
    
    act_values.append(actual_value)
    pred_values.append(fitted_value)
    errors.append(actual_value - fitted_value)
    
    #print(model_fit.summary())
    if index <= 10:
        print(i , ':    ', actual_value, ' & ', fitted_value)

In [ ]:
dff['pred_values_gonorrhea'] = pred_values
dff['errors_gonorrhea'] = errors

In [ ]:
act_values = []
pred_values = []
errors = []

#fips = fips[:500]

for index, i in enumerate(fips):
    df1 = df[(df['fips']==i) & (df['year']>= 2002)].loc[:,cols]
    df_sum = df1.groupby('year').sum().reset_index()
    df_model = df_sum[["year", "cases_early_non-primary_non-secondary_syphilis"]]
    #print(df_model.shape)
    #df_model['cases_early_non-primary_non-secondary_syphilis'] = signal.detrend(df_model['cases_early_non-primary_non-secondary_syphilis'])
    actual_value = df_model.iloc[-1, df_model.columns.get_loc('cases_early_non-primary_non-secondary_syphilis')]
    df_model = df_model[(df_model["year"] <= 2019)]
    df_plot = df_model
    #print(df_model.head(20))
    #print(actual_value)
    
    model = ARIMA(df_model["cases_early_non-primary_non-secondary_syphilis"], order=(2,2,0))
    model_fit = model.fit()
    
    pred = model_fit.predict(18).to_frame()
    fitted_value = pred.iloc[0][0]
    #print(df['year'].nunique())
    
    act_values.append(actual_value)
    pred_values.append(fitted_value)
    errors.append(actual_value - fitted_value)
    
    #print(model_fit.summary())
    if index <= 10:
        print(i , ':    ', actual_value, ' & ', fitted_value)
    #print(i , ':    ', fitted_value)
    #print(df_plot.head(20))
    #print(df_model.head(20))

In [ ]:
dff['pred_values_early_non-primary_non-secondary_syphilis'] = pred_values
dff['errors_early_non-primary_non-secondary_syphilis'] = errors

In [ ]:
act_values = []
pred_values = []
errors = []

#fips = fips[:500]

for index, i in enumerate(fips):
    df1 = df[(df['fips']==i) & (df['year']>= 2002)].loc[:,cols]
    df_sum = df1.groupby('year').sum().reset_index()
    df_model = df_sum[["year", "cases_primary_and_secondary_syphilis"]]
    #print(df_model.shape)
    #df_model['cases_primary_and_secondary_syphilis'] = signal.detrend(df_model['cases_primary_and_secondary_syphilis'])
    actual_value = df_model.iloc[-1, df_model.columns.get_loc('cases_primary_and_secondary_syphilis')]
    df_model = df_model[(df_model["year"] <= 2019)]
    df_plot = df_model
    #print(df_model.head(20))
    #print(actual_value)
    
    model = auto_arima(df_model["cases_primary_and_secondary_syphilis"], trace=False, error_action='ignore', suppress_warnings=True)
    model.fit(df_model["cases_primary_and_secondary_syphilis"])
    
    pred = model.predict(n_periods = 1)
    fitted_value = pred[0]
    #print(df['year'].nunique())
    
    act_values.append(actual_value)
    pred_values.append(fitted_value)
    errors.append(actual_value - fitted_value)
    
    #print(model_fit.summary())
    if index <= 10:
        print(i , ':    ', actual_value, ' & ', fitted_value)
    #print(i , ':    ', fitted_value)
    #print(df_plot.head(20))
    #print(df_model.head(20))

In [ ]:
dff['pred_values_primary_and_secondary_syphilis'] = pred_values
dff['errors_early_primary_and_secondary_syphilis'] = errors

In [ ]:
dff.head(20)

In [ ]:
dff.head(10)

In [ ]:
pop = pd.read_csv('../Data/Final/STI0220_cov0220.csv', index_col = 0).loc[:, ['total_pop', 'fips', 'year']]
pop = pop[pop['year']==2020]
dfe = pd.merge(dff, pop, on = ['fips'], how = 'outer')
dfe.head(10)

In [ ]:
cols2 = ['gonorrhea','chlamydia','early_non-primary_non-secondary_syphilis',
        'early_primary_and_secondary_syphilis']

for i in cols2:
    dfe['adj_errors_' + i] = dfe['errors_' + i]/dfe['total_pop']

dfe.drop(columns = 'year', inplace = True)

In [ ]:
dfe.head(10)

In [ ]:
dfe.to_csv('../Data/Final/model1_pred_vals_errors.csv')